In [1]:
from tqdm import tqdm

## 1. Pre-train on CIFAR-10

In [2]:
'''Train CIFAR10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cifar10.t7')
        best_acc = acc


for epoch in range(start_epoch, 350):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


  0%|          | 0/392 [00:00<?, ?it/s]


Epoch: 0


Acc: 47.020%:  99%|█████████▉| 100/101 [00:02<00:00, 44.70it/s]


Saving..


...

  0%|          | 0/392 [00:00<?, ?it/s]


Epoch: 348


  0%|          | 0/392 [00:00<?, ?it/s]


Epoch: 349


Acc: 96.544%: 100%|█████████▉| 391/392 [00:31<00:00, 12.95it/s]


testing accuracy: >90%

## Train on CINIC-10

- train: train set
- valid: valid set

In [5]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.0.t7')#'/tmp/work/checkpoint/ckpt.cifar10.t7')
net.load_state_dict(checkpoint['net'])
#start_epoch = checkpoint['epoch']
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 150, 300], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.0.t7')
        best_acc = acc


for epoch in range(start_epoch, 501):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..


  0%|          | 0/704 [00:00<?, ?it/s]

==> Building model..
==> Resuming from checkpoint..

Epoch: 0


  0%|          | 0/704 [00:00<?, ?it/s]

Saving..

Epoch: 1


...

  0%|          | 0/704 [00:00<?, ?it/s]


Epoch: 498


  0%|          | 0/704 [00:00<?, ?it/s]


Epoch: 499


Acc: 80.233%: 100%|██████████| 704/704 [01:25<00:00,  8.23it/s]


## Train on full CINIC-10

- train: train set + valid set
- valid: test set

In [ ]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_train)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.0.t7')
net.load_state_dict(checkpoint['net'])
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 250, 450], gamma=0.2)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)+len(validloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)
        for batch_idx, (inputs, targets) in enumerate(validloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.1.t7')
        best_acc = acc

for epoch in range(start_epoch, 601):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
==> Building model..


  0%|          | 0/1408 [00:00<?, ?it/s]

==> Resuming from checkpoint..

Epoch: 0


  0%|          | 0/1408 [00:00<?, ?it/s]

Saving..

Epoch: 1


...

  0%|          | 0/1408 [00:00<?, ?it/s]


Epoch: 191


  0%|          | 0/1408 [00:00<?, ?it/s]


Epoch: 193


Acc: 77.532%:  73%|███████▎  | 1029/1408 [01:40<00:35, 10.71it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Acc: 77.680%:   9%|▊         | 123/1408 [00:11<01:59, 10.77it/s]

## Model Deployment

In [1]:
_name = 'cinic10.1'
src_path = './checkpoint/ckpt.{}.t7'.format(_name)
dest_path = './checkpoint/mobileNetV2.{}.pth'.format(_name)

In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load(src_path)
net.load_state_dict(checkpoint['net'])

==> Building model..
==> Resuming from checkpoint..


In [3]:
net = net.module

In [4]:
state = {
    'net': net.state_dict(),
    'acc': checkpoint['acc'],
    'epoch': checkpoint['epoch'],
}

torch.save(state, dest_path)

In [4]:
# Export model
print('==> Exporting model..')
dummy_input = Variable(torch.randn(32, 3, 32, 32), requires_grad=False).cuda()
torch.onnx.export(net, dummy_input, "./checkpoint/mobileNetV2.{}.onnx".format(_name), verbose=True)

==> Exporting model..
graph(%0 : Float(32, 3, 32, 32)
      %1 : Float(32, 3, 3, 3)
      %2 : Float(32)
      %3 : Float(32)
      %4 : Float(32)
      %5 : Float(32)
      %6 : Long()
      %7 : Float(32, 32, 1, 1)
      %8 : Float(32)
      %9 : Float(32)
      %10 : Float(32)
      %11 : Float(32)
      %12 : Long()
      %13 : Float(32, 1, 3, 3)
      %14 : Float(32)
      %15 : Float(32)
      %16 : Float(32)
      %17 : Float(32)
      %18 : Long()
      %19 : Float(16, 32, 1, 1)
      %20 : Float(16)
      %21 : Float(16)
      %22 : Float(16)
      %23 : Float(16)
      %24 : Long()
      %25 : Float(16, 32, 1, 1)
      %26 : Float(16)
      %27 : Float(16)
      %28 : Float(16)
      %29 : Float(16)
      %30 : Long()
      %31 : Float(96, 16, 1, 1)
      %32 : Float(96)
      %33 : Float(96)
      %34 : Float(96)
      %35 : Float(96)
      %36 : Long()
      %37 : Float(96, 1, 3, 3)
      %38 : Float(96)
      %39 : Float(96)
      %40 : Float(96)
      %41 : Float(96)
    

## Testing

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import sys
data_dir = "/tmp/work/data/CINIC-10/test/"
assert data_dir is not None, "No data directory"

from models import *
checkpoint = torch.load('../mobileNetV2.cinic10.0.pth')
model = MobileNetV2()
model.load_state_dict(checkpoint['net'])
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def inference(model, testloader):
    total = 0
    correct = 0
    model.to(device)
    model = torch.nn.DataParallel(model)
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets.cuda()).sum().item()
    acc = 100.*correct/total
    print(acc)
    return acc

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
inference(model, testloader)